encoder will reduce the dimention/size of the image(data), but increase the features of each pixels of images. We keep reducing the size of each images but keep incerease the features of each pixels of image.this means each pixels will capture more data about the area it belongs. Each pixel will be represented by the 3 channels of RGB. During this process number of pixelis diminishing but each pixel capturing more information.

the job of encoder and decoder of vae is to encode and image or noise to a compressed version of image or noise. So that then we can take the latent and run it to the unet and then after the last step of the dinoissification we take this compressed version or latent and we pass it to the decoder and get the output image.

In [ ]:
#imoorting libraries
import torch
from torch import nn
from torch.nn import functional as F
from decoder import VAE_AttentionBock, VAE_ResidualBock



class VAE_Encoder(nn.Sequential):
#this will inheret from sequential module. basically our encoder is a sequence of super modules.
#It's actually a sequence of super modules, each module reduce the dimention size but increase the features of the data
    def __init__(self):
        super().__init__(
            #(batch_size,channels, height, width)=>(batch_size, 128, height, width)
            nn.conv2D(3, 128, kernel_size=3,padding=1),
#(initially we will have three channels and we convet into 128 channels, with kernel size 3 and padding 1)we start it as batch size define it as 3 channels, image will have a height, width of 512*512 . The convolution will convert into 128 with the same heigt and width, why in this case the height and width didnt change? =>padding

            #(Batch size, 128, height, weight)=>(batch_size, 128, height, width)
            VAE_ResidulBlock(128,128), #this residual block in a combination of bunch of convolution and nomaliation), (128,128)=>how many input channels and output channel ,in residual block we dint change the size of the image)
            #(Batch size, 128, height, weight)=>(batch_size, 128, height, width)
            VAE_ResidulBlock(128,128),
            #(Batch size, 128, height, weight)=>(batch_size, 128, height/2, width/2)
            nn.Conv2D(128,128, kernel_size=3, stride=2, padding=0),#the input channels of the conv is 128 because the output of the last block is 128, so now the size of the image is half of previous size, height width reduced because of stride=2, )
            #(Batch size, 128, height/2, weight/2)=>(batch_size, 256, height/2, width/2)
            VAE_ResidulBlock(128,256), # we just incerese the number of features , didnt increase the size)
            #(Batch size, 128, height/2, weight/2)=>(batch_size, 256, height/2, width/2)
            VAE_ResidulBlock(256,256),
            #(Batch size, 256, height/2, weight/2)=>(batch_size, 256, height/4, width/4)
            nn.Conv2D(256,256, kernel_size=3, stride=2, padding=0),
            #(Batch size, 256, height/4, weight/4)=>(batch_size, 256, height/4, width/4)
            VAE_ResidulBlock(256,512),
            #(Batch size, 256, height/4, weight/4)=>(batch_size, 256, height/4, width/4)
            VAE_ResidulBlock(512,512),
            #(Batch size, 256, height/4, weight/2)=>(batch_size, 256, height/8, width/8)
            nn.Conv2D(512,512, kernel_size=3, stride=2, padding=0),

            VAE_ResidulBlock(512,512),

            VAE_ResidulBlock(512,512),
            #(Batch size, 512, height/8, weight/8)=>(batch_size, 256, height/8, width/8)
            VAE_ResidulBlock(512,512),

            VAE_AttentionBlock(512,512), # this attention block makes all pixel relate to each others, each pixel will relate to each other globally while conv relate pixels locally whose are close to the corresponding pixel)
            #(Batch size, 512, height/8, weight/8)=>(batch_size, 256, height/8, width/8)
            VAE_ResidulBlock(512,512),
            #(Batch size, 512, height/8, weight/8)=>(batch_size, 256, height/8, width/8
            nn.GroupNorm(32,512),

            nn.SiLU(),# practically this activation function works better so they used this
            #(Batch size, 512, height/8, weight/8)=>(batch_size, 8, height/8, width/8
            nn.Conv2D(512,8, kernel_size=3, padding=1),
            #(Batch size, 8, height/8, weight/8)=>(batch_size, 8, height/8, width/8
            nn.Conv2D(8,8, kernel_size=1,padding=0), #size doesnt change because kernel size

in decoder , we will reduce the number of channels and increase the size of the image.
here we need to build three blocks, attention block , residual block and decoder block, we build attention block in a sperate class

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from attention import SelfAttention

#to define attention we need to define self attention block, we did it after residual block
class VAE_AttentionBlock(nn.Module):
    def __init__(self, channels:int):
        super().__init__()
        self.groupnorm=nn.GroupNorm(32,channels)#channels is always 32 in stable diffusion
        self.attention=SelfAttention(1, channels) # attention block has a group normalization and also a attention which is self attention

    def forward(self,x:torch.Tensor)->torch.Tensor:
        #x: (Batch_size, features, Height, Width)

        #again we make a residual connection
        residue=x

        #first thing we do is we extract the shape
        n,c,h,w=x.shape #n=batch size, c= number of channels, height, width


        #then we will self attention among all the pixels of this image
        #(batch_size, featres, height, width)->(batch_size, features, h*w)
        x=x.view(n,c,h*w) #now we have a sequence, where each item represents a pixel bcz we do h*w

        #then we transpose it like this
        #(batch_size, featres, height*width)->(batch_size,h*w, features)
        x=x.transpose(-1,-2)
        #then we do attention
        x=self.attention(x)

        #(batch_size, height*width, features)->(batch_size,features, h*w)
        x=x.transpose(-1,-2)#then we do inverse transformation, because we transpose before to do attention now we transpose back

        #then we again remove the multiplication
        #we go from here (batch_size, features, h*w)->(batch_size, features, height, width)
        x=x.view(n,c,h,w)

        #then we add the residual connection
        x+residue #residual connection wil not change the size of input
        #and we return x
        return x

        return x+residue

class VAE_ResidualBock(nn.module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.groupnorm_1=nn.GroupNorm(32,in_channels)
        self.conv_1=nn.Conv2D(in_channels,in_channels, kernel_size=3, padding=1)


        self.groupnorm_2=nn.GroupNorm(32,in_channels)
        self.conv_2=nn.Conv2D(in_channels,in_channels, kernel_size=3, padding=1)
#skip connection part, skip connection basically means u take input ,skip some layers, n then u connect it with output of last layer)
#we also need this residual connection if the two channels are different we need to create another intermediate layer)
        if in_channels==out_channels:
            self.residual=nn.Identity()
        else:
            self.residual=nn.Conv2D(in_channels, out_channels, kernel_size=1, padding=0)

    def forward(self,x:torch.Tensor)->torch.Tensor:
         #x:(Batch_size, In_channels,Height, Width)

         residue=x
         x=self.groupnorm_1(x)
         x=F.silu(x)
         x=self.conv_1(x)
         x=self.groupnorm_2(x)
         x=F.silu(x)
         x=self.conv_2(x)
         return x+self.residual_layer(residue) #if the number of output channels are not equal to input channels we can not do x+residue, because dimensions will not match
         #so we create [self.residual=nn.Conv2D(in_channels, out_channels, kernel_size=1, padding=0)] layer to convert the input channel to the output channel of x such that
         #sum can be done ,so we apply (residual_layer of rasidue) instead of residue



we built attention block before now we built self attention, there are two types of attention in stable diffusion e.g. self and cross attention, we now built it in different class

In [ ]:
from re import M
from _typeshed import Self
import torch
from torch import nn
from torch.nn import functional as F
import math


class SelfAttention(nn.Module):

    def __init__(self, n_heads:int, d_embed:int, in_proj_bias=True, out_proj_bias=True): #we have the number of head and embeding(token in transformer model we use in pixel of images, we have also the bias for w matrices),
        super().__init__()


        #lets define the w matrices w_q,w_k,w_v
        self.in_proj=nn.Linear(d_embed, 3*d_embed, bias=in_proj_bias) #we built a big linear layer instead of build them sepertely in_proj_bias means in projection bias

        self.out_proj=nn.Linear(d_embed, d_embed, bias=out_proj_bias)
         #then we set nmber of heads
        self.n_heads=n_heads
        #then we set dimension of each head, d_head means basically if we have multihead, each head will watch a part of the embading of each token
        self.d_head=d_embed//n_heads

        #lets implement the forward
        def forward(self,x:torch.Tensor, casual_mask=False)->torch.Tensor: #we can also apply a mask, mask is the way to avoid letting token that comes after it, but only with the token that comes before it
        #x:(Batch_size, Seq_Len,Dim)

        #first we extract the shape
        input_shape=x.shape
        batch_size, sequence_length, d_embed=input_shape

        #then we convert it into another shape
        #(Batch_size, Seq_Len, Dim)->(Batch_size, Seq_Len, 3*Dim)
        intermim_shape=(batch_size, sequence_length, self.n_heads, self.d_head)

        #then we apply the q,k,v
        #(Batch_size, seq_len, Dim) -> (Batch_size, seq_len,Dim*3)->3 tensors of shape(Batch_size, Seq_len,Dim)
        q, k, v=self.in_proj(x).chunk(3,dim=-1) #


        #now we can split the q,k,v according to the number of heads
        #(Batch_size, Seq_Len, Dim)->(Batch_size, Seq_Len, n_heads,H, Dim/H)->(Batch_size,H, Seq_Len, Dim/H) (due to transpose)
        #(Batch_size, Seq_Len, n_heads,H, Dim/H)means it head will watch the full sequence but only a part of embeding of each token (in this case pixel)
        q=q.view(intermim_shape).transpose(1,2)
        k=k.view(intermim_shape).transpose(1,2)
        v=v.view(intermim_shape).transpose(1,2)

        #then we calculate the attention according to formula

        weight=q@k.transpose(-1,-2)
        if casual_mask:
            #mask where the upper tringle(above the principal diagonal) is made up of 1
            mask=torch.ones_like(weight,dtype=torch.bool).triu(1)
            weight.masked_fill_(mask, -torch.inf)

        weight=weight/math.sqrt(self.d_head)
        #then we apply softmax
        weight=F.softmax(weight, dim=-1)


        output=weight@v
        output = output.transpose(1, 2)
        output = output.reshape(input_shape)
        output = self.out_proj(output)
        return output